# Zadatak 3.

U tekstualnom fajlu je dat **težinski** graf gradova Srbije. Napisati program koji pronalazi najkraći put između dva izabrana grada i ispisuje ga.
U tekstualnom fajlu `edges` se nalaze ivice grafa sa njihovim težinama, dok u tekstualnom fajlu `nodes` piše koji grad je predstavljen kojim čvorom u grafu.

In [25]:
import math

def createAdjacencyList():
  with open("graphWeighted", "r") as f:
    line = f.readline()
    line = line.strip('\n')
    data = line.split(' ')
    n = int(data[0])
    m = int(data[1])

    adjList = {}
    for i in range(1, n + 1):
      adjList[i] = []

    for line in f:
      line = line.strip('\n')
      data = line.split(' ')
      u = int(data[0])
      v = int(data[1])
      w = int(data[2])

      adjList[u].append((v, w))
      adjList[v].append((u, w))

    return adjList

def dijkstra(graph, start, end):
  visited = [False] * (len(graph) + 1)
  distances = [1000] * (len(graph) + 1)
  parents = [None] * (len(graph) + 1)

  distances[start] = 0
  queue = [(start, 0)]

  while len(queue) > 0:
    queue.sort(key=lambda x: x[1])

    current, currDistance = queue.pop(0)
    visited[current] = True
    for neighbour, weight in graph[current]:
      if not visited[neighbour] and currDistance + weight <= distances[neighbour]:
        distances[neighbour] = currDistance + weight
        parents[neighbour] = current
        queue.append((neighbour, currDistance + weight))

  path = []
  it = end
  while parents[it] != None:
    path.append(it)
    it = parents[it]
  path.append(it)
  path = path[::-1]

  return path, distances[end]

def heuristic(node1, node2):
    lat1 = math.radians(node1[0])
    lon1 = math.radians(node1[1])
    lat2 = math.radians(node2[0])
    lon2 = math.radians(node2[1])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Radius of Earth in kilometers (use 3956 for miles)
    R = 6371.0

    # Calculate the distance
    distance = R * c

    return distance


def astar(graph, start, end, nodeCoordinates):
  visited = [False] * (len(graph) + 1)
  distances = [1000] * (len(graph) + 1)
  parents = [None] * (len(graph) + 1)

  distances[start] = 0
  queue = [(start, 0)]
  while len(queue) > 0:
    queue.sort(key=lambda x: x[1])

    current, _ = queue.pop(0)
    currDistance = distances[current]

    if current == end:
      break

    if visited[current]:
      continue
    else:
      visited[current] = True

    for neighbour, weight in graph[current]:
      if currDistance + weight < distances[neighbour]:
        distances[neighbour] = currDistance + weight
        parents[neighbour] = current
        queue.append((neighbour, distances[neighbour] + heuristic(nodeCoordinates[end], nodeCoordinates[neighbour])))

  path = []
  it = end
  while parents[it] != None:
    path.append(it)
    it = parents[it]
  path.append(it)
  path = path[::-1]

  return path, distances[end]

if __name__ == "__main__":
  adjList = createAdjacencyList()

  nodeCoordinates = {}
  with open("nodesCoordinates", "r") as f:
    for line in f:
      line = line.strip('\n')
      line = line.split(' ')
      nodeCoordinates[int(line[0])] = (float(line[2]), float(line[3]))

  path, distance = astar(adjList, 1, 4, nodeCoordinates)
  print("Path is: ", path)


Path is:  [1, 12, 14, 4]
